<a href="https://colab.research.google.com/github/sinjy1203/deep-learning/blob/master/mnist_early_stopping(0to4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import tensorflow as tf
import numpy as np
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [0]:
x_train, x_test = x_train.astype(np.float32).reshape(-1, 28*28), x_test.astype(np.float32).reshape(-1, 28*28)
y_train, y_test = y_train.astype(np.int32), y_test.astype(np.int32)
from sklearn.model_selection import train_test_split
x_train, x_val = train_test_split(x_train, test_size=0.2, random_state=42)
y_train, y_val = train_test_split(y_train, test_size=0.2, random_state=42)

In [59]:
a = np.array([[1,2,3],
              [4,5,6],
              [7,8,9]])
np.delete(a, (0, 1), axis=0)

array([[7, 8, 9]])

In [0]:
not_idx = []
not2_idx = []
for idx, i in enumerate(y_train):
  if (i >= 5) and (i <= 9):
    not_idx.append(idx)
for idx, i in enumerate(y_val):
  if (i >= 5) and (i <= 9):
    not2_idx.append(idx)
y_train = np.delete(y_train, (not_idx), axis=0)
x_train = np.delete(x_train, (not_idx), axis=0)
y_val = np.delete(y_val, (not2_idx), axis=0)
x_val = np.delete(x_val, (not2_idx), axis=0)

In [77]:
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)
import time
from functools import partial

def batch_shuffle(x, y, batch_size):
  rnd_idx = np.random.permutation(len(x))
  n_batch = len(x) // batch_size
  for batch_idx in np.array_split(rnd_idx, n_batch):
    batch_x = x[batch_idx]
    batch_y = y[batch_idx]
    yield batch_x, batch_y

x = tf.placeholder(dtype=tf.float32, shape=(None, 28*28), name='x')
y = tf.placeholder(dtype=tf.int64, shape=(None), name='y')
he_init = tf.variance_scaling_initializer()
layer = partial(tf.layers.dense, units=100, activation=tf.nn.elu, kernel_initializer=he_init)

layer1 = layer(x)
layer2 = layer(layer1)
layer3 = layer(layer2)
layer4 = layer(layer3)
layer5 = layer(layer4)
logits = tf.layers.dense(layer5, 5, kernel_initializer=he_init)

pred = tf.nn.softmax(logits)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(0.001)
training_op = optimizer.minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, axis=1), y), dtype=tf.float32))

n_epochs = 101
batch_size = 100
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# with tf.Session() as sess:
#   sess.run(init)
#   for epoch in range(n_epochs):
#     for xs, ys in batch_shuffle(x_train, y_train, batch_size):
#       sess.run(training_op, feed_dict={x: xs, y: ys})
#     if epoch % 10 == 0:
#       acc = accuracy.eval(feed_dict={x: x_val, y: y_val})
#       print(epoch, "   ", acc)
#   # acc1 = accuracy.eval(feed_dict={x: x_test, y: y_test})
#   # print("최종 정확도: ", acc1)

#######조기종료#########
epoch = 0
best_loss = np.infty
max_count = 10
mis_count = 0
with tf.Session() as sess:
  sess.run(init)
  while(True):
    epoch += 1
    for xs, ys in batch_shuffle(x_train, y_train, batch_size):
      sess.run(training_op, feed_dict={x: xs, y: ys})   # 트레이닝
    cost, acc = sess.run([loss, accuracy], feed_dict={x: x_val, y: y_val})   #val의 손실과 정확도
    # if epoch % 10 == 0:
    #   print("{}번째 손실: {:.6f}  정확도: {:.6f}".format(epoch, cost, acc))    #일정 간격 현상황 프린트
    if cost < best_loss:
      save_path = saver.save(sess, "/content/gdrive/My Drive/tmp/my_model_0to4.ckpt")
      best_loss = cost
      best_accuracy = acc   
      mis_count = 0                         # 손실이 줄어들때
    else:                                          # 손실이 줄어들지 않을때
      mis_count += 1
      if mis_count == 1:
        print("위기상황!!!!!!")
      if mis_count <= max_count:                  # 손실이 줄어들지 않은경우가 5번 이하일때
        print("{}번째 손실: {:.6f}  정확도: {:.6f}  최선의 손실: {:.6f}  최선의 정확도: {:.6f}".format(mis_count, cost, acc, best_loss, best_accuracy))
      else:                                   # 손실이 줄어들지 않은경우가 5번 넘어가면 아웃
        break

    
  # acc1 = accuracy.eval(feed_dict={x: x_test, y: y_test})
  # print("최종 정확도: ", acc1)

위기상황!!!!!!
1번째 손실: 0.081094  정확도: 0.978404  최선의 손실: 0.070283  최선의 정확도: 0.979888
위기상황!!!!!!
1번째 손실: 0.095904  정확도: 0.979888  최선의 손실: 0.057069  최선의 정확도: 0.985493
2번째 손실: 0.070595  정확도: 0.983185  최선의 손실: 0.057069  최선의 정확도: 0.985493
위기상황!!!!!!
1번째 손실: 0.059685  정확도: 0.987306  최선의 손실: 0.046795  최선의 정확도: 0.987801
2번째 손실: 0.080431  정확도: 0.984998  최선의 손실: 0.046795  최선의 정확도: 0.987801
3번째 손실: 0.080695  정확도: 0.985328  최선의 손실: 0.046795  최선의 정확도: 0.987801
4번째 손실: 0.050412  정확도: 0.988955  최선의 손실: 0.046795  최선의 정확도: 0.987801
5번째 손실: 0.057278  정확도: 0.989285  최선의 손실: 0.046795  최선의 정확도: 0.987801
6번째 손실: 0.075875  정확도: 0.981042  최선의 손실: 0.046795  최선의 정확도: 0.987801
7번째 손실: 0.059379  정확도: 0.987141  최선의 손실: 0.046795  최선의 정확도: 0.987801
8번째 손실: 0.058692  정확도: 0.987801  최선의 손실: 0.046795  최선의 정확도: 0.987801
9번째 손실: 0.071260  정확도: 0.985493  최선의 손실: 0.046795  최선의 정확도: 0.987801
10번째 손실: 0.065676  정확도: 0.987306  최선의 손실: 0.046795  최선의 정확도: 0.987801
